In [1]:
import pandas as pd

# Import dataset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2024-3-31_HRA/2024-4-24_preprocessedUlasan.csv')

In [10]:
df.info()
df = df.dropna(subset=['good_translate'])
df.info()
df = df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2377 entries, 0 to 2376
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Location        2377 non-null   object 
 1   AccountName     2372 non-null   object 
 2   ReviewCount     2377 non-null   float64
 3   ulasan          2374 non-null   object 
 4   translate       2377 non-null   object 
 5   Row_Num         2377 non-null   int64  
 6   good_translate  2375 non-null   object 
 7   split_sentence  2377 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 148.7+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 2375 entries, 0 to 2376
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Location        2375 non-null   object 
 1   AccountName     2370 non-null   object 
 2   ReviewCount     2375 non-null   float64
 3   ulasan          2372 non-null   object 
 4   tran

In [2]:
df_split_sentence = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2024-3-31_HRA/2024-4-25_sentimentResultUlasan_split_sentence.csv')

In [3]:
df_good_translate = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2024-3-31_HRA/2024-4-25_sentimentResultUlasan_good_translate.csv')

# Find opinion

## Using dependency rule
https://achyutjoshi.github.io/aspect_extraction/aspectextraction

In [ ]:
import spacy
import pandas as pd
import os
import sys
!python -m spacy download en_core_web_lg

In [6]:
nlp=spacy.load("en_core_web_lg")

In [38]:
#@title Function: Dependency Rule
def apply_extraction(row,nlp):
    review_body = row['good_translate']
    review_id = row['Row_Num']

    doc=nlp(review_body)

    ## FIRST RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    ## RULE = M is child of A with a relationshio of amod
    rule1_pairs = []
    for token in doc:
        if token.dep_ == "amod":
            rule1_pairs.append((token.head.text, token.text))
            #return row['height'] * row['width']


    ## SECOND RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    #Direct Object - A is a child of something with relationship of nsubj, while
    # M is a child of the same something with relationship of dobj
    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule2_pairs = []
    for token in doc:
        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text
            if(child.dep_ == "dobj"):
                M = child.text
        if(A != "999999" and M != "999999"):
            rule2_pairs.append((A, M))


    ## THIRD RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    #Adjectival Complement - A is a child of something with relationship of nsubj, while
    # M is a child of the same something with relationship of acomp
    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule3_pairs = []

    for token in doc:

        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text

            if(child.dep_ == "acomp"):
                M = child.text

        if(A != "999999" and M != "999999"):
            rule3_pairs.append((A, M))

    ## FOURTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect

    #Adverbial modifier to a passive verb - A is a child of something with relationship of nsubjpass, while
    # M is a child of the same something with relationship of advmod

    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule4_pairs = []
    for token in doc:


        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubjpass"):
                A = child.text

            if(child.dep_ == "advmod"):
                M = child.text

        if(A != "999999" and M != "999999"):
            rule4_pairs.append((A, M))


    ## FIFTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect

    #Complement of a copular verb - A is a child of M with relationship of nsubj, while
    # M has a child with relationship of cop

    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule5_pairs = []
    for token in doc:
        children = token.children
        A = "999999"
        buf_var = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text

            if(child.dep_ == "cop"):
                buf_var = child.text

        if(A != "999999" and buf_var != "999999"):
            rule3_pairs.append((A, token.text))

    aspects = []
    aspects = rule1_pairs + rule2_pairs + rule3_pairs +rule4_pairs +rule5_pairs
    dic = {"Row_Num" : review_id , "Aspect_Pairs" : aspects}
    return dic

In [42]:
reviews = df[['Row_Num', 'good_translate']]

review_decomp = reviews.apply(lambda row: apply_extraction(row,nlp), axis=1)

In [106]:
import ast
import re
import json

# Define column names
columns = ['Row_Num','sentence','aspect','sentiment','pairs_dependency_rule']
# Create an empty DataFrame with specified column names
df_sentiment_opinion = pd.DataFrame(columns=columns)

for pairs in review_decomp:
  Row_Num = pairs['Row_Num']
  aspect_pairs_list = pairs['Aspect_Pairs']

  sentence = df_good_translate.loc[df_good_translate['Row_Num'] == Row_Num, 'sentence'].iloc[0]
  aspect = ast.literal_eval(df_good_translate.loc[df_good_translate['Row_Num'] == Row_Num, 'aspect'].iloc[0])
  sentiment = df_good_translate.loc[df_good_translate['Row_Num'] == Row_Num, 'sentiment'].iloc[0]

  matched_pair = []
  for i in aspect_pairs_list:
    pattern_i = r'\b' + re.escape(i[0].lower()) + r'\b'
    for j in aspect:
      pattern_j = r'\b' + re.escape(j.lower()) + r'\b'
      if re.search(pattern_j, i[0].lower()) or re.search(pattern_i, j.lower()):
        matched_pair.append([str(j), str(i[0]), str(i[1])])

  # Append to df_sentiment_opinion
  json_data = {
    "Row_Num": Row_Num,
    "sentence": sentence,
    "aspect": aspect,
    "sentiment": sentiment,
    "pairs_dependency_rule": matched_pair
    }

  # Convert the dictionary to a JSON string
  json_string = json.dumps(json_data)

  df_sentiment_opinion = df_sentiment_opinion._append(json_data, ignore_index=True)

In [107]:
df_sentiment_opinion

,Row_Num,sentence,aspect,sentiment,pairs_dependency_rule
0,0,"During Ramadan and Eid , the services should h...","[services, cashier, services]","['Negative', 'Negative', 'Negative']","[[services, services, faster], [services, serv..."
1,1,I have been a regular customer here . I am dis...,"[service, employee, staff]","['Negative', 'Negative', 'Negative']","[[staff, staff, upset]]"
2,2,The security guard named Lukman and Namnung ha...,[security guard],['Negative'],"[[security guard, guard, attitudes]]"
3,3,Please add a parking area for cars . I was sur...,[parking area],['Negative'],[]
4,4,The store is very hot . They should be aware t...,"[cashiers, queue]","['Negative', 'Negative']","[[queue, queue, long]]"
...,...,...,...,...,...
2370,4746,Complete .,[],[],[]
2371,4747,Manna Kampus Mini Dira is a minimarket that se...,"[staff, store, prices]","['Positive', 'Positive', 'Positive']","[[staff, staff, friendly], [store, store, clea..."
2372,4748,No need to rush to Manna Kampus Jl . Godean is...,[],[],[]
2373,4749,"The service is really satisfying , the price i...","[service, price, goods]","['Positive', 'Positive', 'Positive']","[[service, service, satisfying], [price, price..."


**Checkpoint 6**
*   Dataframe saved as 2024-4-25_sentimentOpinion_good_translate_dependency_rule.csv

In [155]:
# Save dataframe df_sentiment_opinion
df_sentiment_opinion.to_csv('/content/drive/MyDrive/Colab Notebooks/2024-3-31_HRA/2024-4-25_sentimentOpinion_good_translate_dependency_rule.csv', index=False)

# Not Used

In [117]:
filtered_df_sentiment_opinion = df_sentiment_opinion[(df_sentiment_opinion['pairs_dependency_rule'].apply(len) == 0) & (df_sentiment_opinion['sentiment'] != "[]")]

In [121]:
filtered_df_sentiment_opinion

,Row_Num,sentence,aspect,sentiment,pairs_dependency_rule
3,3,Please add a parking area for cars . I was sur...,[parking area],['Negative'],[]
11,11,I had a very rude experience with the cashier ...,[cashier],['Negative'],[]
14,14,"On February 2 , 2022 at 4 : 10 PM , I went to ...",[staff],['Negative'],[]
16,16,I am annoyed . Even though I didn ' t buy Bodr...,[cashier],['Negative'],[]
17,17,"Very bad , I bought a fan , the display item w...",[display],['Negative'],[]
...,...,...,...,...,...
2339,4705,Incomplete . Looking for Frisian Flag full - c...,"[-, milk]","['Negative', 'Negative']",[]
2342,4708,"Quite sufficient for daily needs , but unfortu...",[cat food],['Negative'],[]
2350,4726,"The price is the same as the one downtown , th...","[price, daily]","['Neutral', 'Positive']",[]
2360,4736,It is cheap and trusted . The breads are also ...,[breads],['Positive'],[]


In [131]:
review_decomp[26]

{'Row_Num': 26, 'Aspect_Pairs': [('they', 'Why')]}

In [128]:
!pip install afinn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53430 sha256=69dba099d7eeb12200983db1d2ceff3977e3c9307a7325ddab88f4b20adc9817
  Stored in directory: /root/.cache/pip/wheels/b0/05/90/43f79196199a138fb486902fceca30a2d1b5228e6d2db8eb90
Successfully built afinn


In [147]:
from afinn import Afinn
import spacy

# Load AFINN lexicon
afinn = Afinn()

for index, row in filtered_df_sentiment_opinion.iterrows():
  # Example sentence
  sentence = row['sentence']
  # Tokenize the sentence
  words = sentence.split()

  # Get sentiment words and their scores
  sentiment_words = []
  for word in words:
    score = afinn.score(word)
    if score != 0:
      sentiment_words.append(word)

  # Iterate over aspect
  aspect = row['aspect']
  for a in aspect:
    target_word = a
    # Tokenize the sentence
    doc = nlp(sentence)

    # Find the token indices of the target word and words in the list
    target_word_index = None
    words_indices = {}
    for token in doc:
      if token.text.lower() == target_word:
        target_word_index = token.i
      if token.text.lower() in sentiment_words:
        words_indices[token.text.lower()] = token.i

    # Calculate distances from target word to words in the list
    distances = {sentiment_words: abs(target_word_index - word_index) for word, word_index in words_indices.items()}

    # Find the word with the smallest distance
    nearest_word = min(distances, key=distances.get)

    print(row['Row_Num'], a, nearest_word)


In [153]:
from afinn import Afinn

# Load AFINN lexicon
afinn = Afinn()

# Example sentence
sentence = filtered_df_sentiment_opinion['sentence'][61]
# Tokenize the sentence
words = sentence.split()

# Get sentiment words and their scores
sentiment_words = []
for word in words:
    score = afinn.score(word)
    if score != 0:
        sentiment_words.append(word)

print(sentiment_words)


['sorry', 'Please', 'shame', 'Please', 'rude', 'comfortably', 'Thank']


In [154]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Example sentence and target word
sentence = filtered_df_sentiment_opinion['sentence'][61]
target_word = "staff"

# List of words
words = sentiment_words

# Tokenize the sentence
doc = nlp(sentence)

# Find the token indices of the target word and words in the list
target_word_index = None
words_indices = {}
for token in doc:
    if token.text.lower() == target_word:
        target_word_index = token.i
    if token.text.lower() in words:
        words_indices[token.text.lower()] = token.i

# Calculate distances from target word to words in the list
distances = {word: abs(target_word_index - word_index) for word, word_index in words_indices.items()}

# Find the word with the smallest distance
nearest_word = min(distances, key=distances.get)

print("Nearest word:", nearest_word)


TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'